In [163]:
import re
import json
from pprint import pprint
from yaml import safe_load
import toolz

SOURCES = {
    'youtube.com': 'Youtube',
    'folha.uol.com.br': 'Folha de São Paulo',
    'bbc.com': 'BBC Brasil',
    'g1.globo.com': 'G1',
    'oglobo.globo.com': 'O Globo',
    'terra.com.br': 'Terra',
    'brasil.elpais.com': 'El País',
    'metro1.com.br': 'Metro 1',
    'revistaladoa.com.br': 'Revista Lado A',
    'portaldiariodonorte.com.br': 'Portal Diário do Norte',
}

h1_regex = re.compile(r'^#\s*([^\n]*)')
url_regex = re.compile(r'^https?://(www\.)?([^/]+)/')

In [178]:
def clean_title(st):
    return st.strip('"\' \n')

def youtube_embed(st):
    _, _, ref = st.partition('v=')
    return 'https://www.youtube.com/embed/' + ref

def remove_empty_lines(st):
    return '\n'.join(line for line in st.splitlines() if not line.isspace())

def parse_re(regex, doc):
    m = regex.match(doc)
    if m is None:
        raise TypeError('do not match document')

    i, j = m.span()
    return m.group(1), doc[j:]

def parse_souce(source):
    source = source.strip('<>')
    try:
        domain = url_regex.match(source).group(2)
    except AttributeError:
        raise ValueError(f'invalid source: {source!r}')
    name = SOURCES[domain]
    return {'name': name, 'url': source}

def parse_rant(rant):
    text, _, source = rant.rpartition('<')
    source = source.rstrip('>')
    text = text.strip()
    return {'text': text, 'source': parse_souce(source)}

def parse_event(st):
    st = st.strip()
    event = {}

    # Get title
    lines = toolz.remove(lambda x: not x or x.isspace(), st.splitlines())
    title, *lines = toolz.partitionby(lambda x: x.startswith(' '), lines)
    title = title[0]
    if title.endswith(')'):
        title, _, source = title.partition('(')
        title = title.strip()
        source = source.strip('(')
    else:
        source = None
    event['title'] = title

    try:
        data, description = lines
    except ValueError:
        raise ValueError(f'invalid event:\n{st}')
    data = safe_load('\n'.join(map(str.strip, data)))
    event['image'] = data['imagem']
    event['text'] = description[0]

    if source:
        event['source'] = {'name': source, 'url': data['url']}
    else:
        event['source'] = parse_souce(data['url'])

    return event

def parse_story(doc):
    doc = doc.strip()
    story = {}

    # Split event from data
    title, doc = parse_re(h1_regex, doc)
    title = clean_title(title)
    section, *events = doc.split('\n##')
    story['utter'] = title

    # Process data
    lines = toolz.remove(lambda x: not x or x.isspace(), section.splitlines())
    quote, *other = toolz.partitionby(lambda x: x.startswith(' '), lines)
    *quote, who = map(str.strip, quote)
    story['bible'] = '\n'.join(quote)
    story['ref'] = who.strip(' -')

    # Process yaml
    yaml, *other = other
    if yaml[-1] == 'outras:':
        yaml = yaml[:-1]
    meta = safe_load('\n'.join(yaml))
    story['image'] = meta['foto'] or '/static/generic.jpg'
    story['youtube'] = youtube_embed(meta['video'])

    if other:
        story['rants'] = list(map(lambda x: parse_rant(x.lstrip('- ')), other[0]))

    # Process events
    story['events'] = list(map(parse_event, events))

    return story

In [179]:
stories = list(map(parse_story, open('data.md').read().split('---')))

with open('static/data.json', 'w') as fd:
    json.dump(stories, fd)
    
stories

[{'utter': 'Vamos fuzilar a petralhada',
  'bible': 'Aí Jesus disse: "Guarde a sua espada, pois quem usa a espada será morto por uma espada".',
  'ref': 'Mateus 22:52',
  'image': '/static/generic.jpg',
  'youtube': 'https://www.youtube.com/embed/FYErb6oriiU',
  'rants': [{'text': 'Quanto mais se matar, melhor',
    'source': {'name': 'Youtube',
     'url': 'https://www.youtube.com/watch?v=Ii-Bo9HPAeE'}},
   {'text': 'Violência se combate com violência',
    'source': {'name': 'Youtube',
     'url': 'https://www.youtube.com/watch?v=o8ECr0eDEGo'}},
   {'text': 'Bolsonaro ensina criança a fazer gesto de arma e causa revolta',
    'source': {'name': 'Youtube',
     'url': 'https://www.youtube.com/watch?v=hpep_70CWlw'}}],
  'events': [{'title': 'Mestre de capoeira é morto por eleitor de Bolsonaro após declarar voto no PT',
    'image': 'https://extra.globo.com/incoming/23139253-c7d-8af/w640h360-PROP/xmoa.jpg.pagespeed.ic.jMLoqYFCWZ.jpg',
    'text': 'Mestre Moa do Katendê, de 63 anos, foi 

In [174]:
!cat data.md

# Vamos fuzilar a petralhada

    Aí Jesus disse: "Guarde a sua espada, pois quem usa a espada será morto por uma espada".
        - Mateus 22:52

foto:
video: https://www.youtube.com/watch?time_continue=53&v=FYErb6oriiU
outras:
    - Quanto mais se matar, melhor <https://www.youtube.com/watch?v=Ii-Bo9HPAeE>
    - Violência se combate com violência <https://www.youtube.com/watch?v=o8ECr0eDEGo>
    - Bolsonaro ensina criança a fazer gesto de arma e causa revolta <https://www.youtube.com/watch?v=hpep_70CWlw>


## Mestre de capoeira é morto por eleitor de Bolsonaro após declarar voto no PT

    url: https://www.bbc.com/portuguese/brasil-45806355
    imagem: https://extra.globo.com/incoming/23139253-c7d-8af/w640h360-PROP/xmoa.jpg.pagespeed.ic.jMLoqYFCWZ.jpg

Mestre Moa do Katendê, de 63 anos, foi assassinado com 12 facadas após se posicionar contra Jair Bolsonaro num bar de Salvador

## Mulher é agredida por eleitores de Bolsonaro em Porto Alegre

    url: <https://g1.globo.com/rs/rio-gran